In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
import tsfresh
from pmdarima.arima import ADFTest
from tsfresh import extract_features
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
values = read_csv("../../Time_series_data.csv")

In [3]:
values.head(13)

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
5,2013-06,3036
6,2013-07,2282
7,2013-08,2212
8,2013-09,2922
9,2013-10,4301


In [4]:

values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})
values['Time'] = pd.to_datetime(
    values['Time'], errors='coerce')

In [5]:
freq = pd.infer_freq(values["Time"])

In [6]:
# end_date = pd.to_datetime(x) + pd.DateOffset(months=10)
# print(end_date)

In [7]:
values.set_index('Time', inplace=True)

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    values, test_size=0.2, shuffle=False)

rnn_train = train
rnn_test = test

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(rnn_train)
scaled_train = scaler.transform(rnn_train)
scaled_test = scaler.transform(rnn_test)

In [10]:
from keras.preprocessing.sequence import TimeseriesGenerator

# define generator
n_input = 12
n_features = 1
generator = TimeseriesGenerator(
    scaled_train, scaled_train, length=n_input, batch_size=1)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [11]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu',
            input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()
model.fit(generator, epochs=50)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
72/72 [==============================] - 2s 4ms/step - loss: 0.0618
Epoch 2/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0507
Epoch 3/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0453
Epoch 4/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0468
Epoch 5/50
72/72 [==============================] - 0s 4ms/step - loss: 0.0353
Epoch 6/50
72/72 [=

In [12]:
last_train_batch = scaled_train[-12:]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))


In [13]:
last_train_batch

array([[[0.08636474],
        [0.10742931],
        [0.14558859],
        [0.1755651 ],
        [0.1096978 ],
        [0.19549542],
        [0.21421048],
        [0.0133679 ],
        [0.29555213],
        [0.39301628],
        [0.66993438],
        [0.93194523]]])

In [14]:
model.predict(last_train_batch)

test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

1/1 [==============================] - 0s 237ms/step


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
for i in range(len(rnn_test)):
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]

    # append the prediction into the array
    test_predictions.append(current_pred)
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:, 1:, :], [
                                [current_pred]], axis=1)

1/1 [==============================] - 0s 21ms/step


In [17]:
current_batch

array([[[0.44136667],
        [0.75908643],
        [0.95940435],
        [0.19876505],
        [0.1526933 ],
        [0.14710139],
        [0.1545148 ],
        [0.13947679],
        [0.17857863],
        [0.20309703],
        [0.14643359],
        [0.29306346]]])

In [18]:
true_predictions = scaler.inverse_transform(test_predictions)
rnn_test['Predictions'] = true_predictions

In [23]:
mae = mean_absolute_error(rnn_test['Data'], rnn_test['Predictions'])
mae

0.865292678328497

In [22]:
mse = mean_squared_error(rnn_test['Data'], rnn_test['Predictions'])
mse

1043291.335971005

In [19]:
rnn_test

,Data,Predictions
Time,,
2020-01-01,3934,3543.503460
2020-02-01,3162,3295.515683
2020-03-01,4286,3421.650918
2020-04-01,4676,3626.124056
2020-05-01,5010,3206.401798
2020-06-01,4874,3891.932145
2020-07-01,4633,4163.714855
2020-08-01,1659,2825.011539
2020-09-01,5951,5169.837938
